# Comment Country

- In this notebook, I prepared a batch to identify which country or region each thesis-relevant comment refers to.  
- I used the rewritten comment text to build the batch JSONL file.  
- The prompt asked GPT-4o to extract the name of the country or region mentioned in each comment, if any.  
- To stay within the token limit, I split the batch into two parts before submitting it to OpenAI’s Batch API.  
- Once the outputs were ready, I downloaded the JSONL files and extracted the `custom_id` and country response.  
- I saved the results to a CSV, enabling further analysis of which countries are most frequently mentioned across the comments.


### Importing Libraries

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
import openai
import os

### Creating the Json file

In [8]:
# Loading my Dataset
df = pd.read_csv("Final_Thesis_Merged.csv")

# Output file name
output_jsonl = "batch_country_extraction.jsonl"

with open(output_jsonl, "w", encoding="utf-8") as f:
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Building Gulf State Extraction Batch"):
        comment_id = str(row["Comment_ID"])
        comment = str(row["Rewritten Comment"])
        event = str(row["Event"])

        task = {
            "custom_id": comment_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are assisting a researcher analyzing Gulf-backed sports events."
                    },
                    {
                        "role": "user",
                        "content": f'''Below is a YouTube comment and the related sports event. Identify which Gulf state (e.g., Saudi Arabia, Qatar, UAE) is most associated with this comment. Use the event context to help you.

Comment: "{comment}"

Event: "{event}"

If no Gulf state is clearly implied, respond with an empty string. Otherwise, respond with just one country name: Saudi Arabia, Qatar, or UAE.'''
                    }
                ],
                "temperature": 0,
                "max_tokens": 10
            }
        }

        f.write(json.dumps(task) + "\n")

print(f"Saved: {output_jsonl}")


/var/folders/nv/wkg5hpgj1kl1012mfw_qwby80000gn/T/ipykernel_2232/4066860633.py:6: DtypeWarning: Columns (11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Final_Thesis_Merged.csv")
Building Gulf State Extraction Batch: 100%|█| 71441/71441 [00:07<00:00, 9851.77i

Saved: batch_country_extraction.jsonl


### Splitting the Batch due to Token Limit

In [12]:
input_file = "batch_country_extraction.jsonl"  # Change to your file
output_file_1 = input_file.replace(".jsonl", "_part1.jsonl")
output_file_2 = input_file.replace(".jsonl", "_part2.jsonl")

# Reading the file
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Splitting
mid = len(lines) // 2
part1 = lines[:mid]
part2 = lines[mid:]

# Writing to two new files
with open(output_file_1, "w", encoding="utf-8") as f:
    f.writelines(part1)

with open(output_file_2, "w", encoding="utf-8") as f:
    f.writelines(part2)

print(f"Saved {len(part1)} → {output_file_1}")
print(f"Saved {len(part2)} → {output_file_2}")


Saved 35720 → batch_country_extraction_part1.jsonl
Saved 35721 → batch_country_extraction_part2.jsonl


### Submitting Batches to Openai

In [15]:
openai.api_key = "**************"
client = openai.OpenAI(api_key=openai.api_key)

# Your JSONL batch files (submitted one after the other with no wait)
batch_files = [
    "batch_country_extraction_part1.jsonl",
    "batch_country_extraction_part2.jsonl"
]

# Submitting batches
for i, file_path in enumerate(batch_files):
    try:
        print(f"\nSubmitting batch {i+1}/{len(batch_files)}: {file_path}")

        # Uploading file
        with open(file_path, "rb") as f:
            upload = client.files.create(file=f, purpose="batch")
        print(f"Uploaded: File ID = {upload.id}")

        # Creating batch job
        batch = client.batches.create(
            input_file_id=upload.id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        print(f"Batch submitted: Batch ID = {batch.id}")

    except Exception as e:
        print(f"Failed to submit {file_path}: {e}")



Submitting batch 1/2: batch_country_extraction_part1.jsonl
Uploaded: File ID = file-8JUTCtPxpXYTk929LcpifC
Batch submitted: Batch ID = batch_68163f99dbb8819095abba6692ef4663

Submitting batch 2/2: batch_country_extraction_part2.jsonl
Uploaded: File ID = file-PYuhXk4cesH9mvuEQFbYQh
Batch submitted: Batch ID = batch_68163fa478a8819092324878bdb1b702


### Downloading Json Output from Openai

In [20]:
openai.api_key = "*************" 

# List of output file IDs from completed batches
output_file_ids = [
    "file-5gGugC2T7b6pfk56LfDCKY",
    "file-8GHHh1dZMLRrW7jGLPuVo5"
]

# Downloading each output file
for file_id in output_file_ids:
    file_response = openai.files.content(file_id)

    # Saving the file locally in binary mode
    output_filename = f"{file_id}.jsonl"
    with open(output_filename, "wb") as f:
        for chunk in file_response.iter_bytes():
            f.write(chunk)
    
    print(f"File downloaded: {output_filename}")

File downloaded: file-5gGugC2T7b6pfk56LfDCKY.jsonl
File downloaded: file-8GHHh1dZMLRrW7jGLPuVo5.jsonl


### Converting Json Output to a CSV file

In [29]:
jsonl_files = [
    "file-5gGugC2T7b6pfk56LfDCKY.jsonl",
    "file-8GHHh1dZMLRrW7jGLPuVo5.jsonl"
]

# Valid Gulf states 
valid_countries = {
    "Saudi Arabia",
    "Qatar",
    "UAE"
}

# Storing clean records
records = []

# Processing each file
for jsonl_file in jsonl_files:
    if not os.path.exists(jsonl_file):
        print(f"Missing file: {jsonl_file}")
        continue

    print(f"Processing {jsonl_file}")
    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            comment_id = obj.get("custom_id", "")
            raw_response = obj.get("response", {}).get("body", {}).get("choices", [{}])[0].get("message", {}).get("content", "").strip()

            # Clean response
            country = raw_response.strip('`"').strip()
            if country not in valid_countries:
                country = ""  # Default to empty string if not valid

            records.append({
                "custom_id": comment_id,
                "Country": country
            })

df = pd.DataFrame(records)
df.to_csv("final_country_responses.csv", index=False)

print("\nSaved to: final_country_responses.csv")
print(df["Country"].value_counts(dropna=False))


Processing file-5gGugC2T7b6pfk56LfDCKY.jsonl
Processing file-8GHHh1dZMLRrW7jGLPuVo5.jsonl

Saved to: final_country_responses.csv
Country
Qatar           49848
Saudi Arabia    16478
UAE              3996
                 1119
Name: count, dtype: int64


### Merging Country Classfication in to Main Dataset

In [3]:
df_final = pd.read_csv("Final_Thesis_Merged.csv")

# Load the final country responses
df_country = pd.read_csv("final_country_responses.csv")

# Merge based on Comment_ID and custom_id
df_final = df_final.merge(
    df_country,
    how="left",
    left_on="Comment_ID",
    right_on="custom_id"
)

# Dropping the redundant custom_id column
df_final = df_final.drop(columns=["custom_id"])

# Overwriting the original thesis file
df_final.to_csv("Final_Thesis_Merged.csv", index=False)

/var/folders/nv/wkg5hpgj1kl1012mfw_qwby80000gn/T/ipykernel_818/1785757420.py:4: DtypeWarning: Columns (11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("Final_Thesis_Merged.csv")
